# Inception/GoogleNet

### From Paper
- https://arxiv.org/pdf/1409.4842.pdf


## Points from Paper
- *“we need to go deeper”*
- In inception module, 1×1convolutions are used to compute reductions before the expensive 3×3 and 5×5 convolutions.


## Inception Module

![](./inception_arch.png)

In [1]:
import torch
from torch import nn, optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import torchvision

## Setting Device

In [2]:
available_device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"{available_device} is available")
device = torch.device(device=available_device)
device


cuda is available


device(type='cuda')

In [101]:
class ConvBlock(nn.Module):
    def __init__(self, in_channel, out_channel, **kwargs):
        super(ConvBlock, self).__init__()
        self.conv = nn.Conv2d(in_channel, out_channel, **kwargs )
        self.relu = nn.ReLU()
        self.localRespNorm = nn.LocalResponseNorm(out_channel)
    def forward(self, x):
        x = self.relu(self.conv(x))
        return x

In [44]:
x = torch.randn(2,192,28,28)
x.shape

torch.Size([2, 192, 28, 28])

In [46]:
conv1 = ConvBlock(192,64,kernel_size=1)
y = conv1(x)
y.shape

torch.Size([2, 64, 28, 28])

In [102]:
class Inception_Module(nn.Module):
    def __init__(self, in_channel, out1x1, re_out3x3, out3x3, re_out5x5, out5x5,  out_pool):
        super(Inception_Module, self).__init__()
        self.branch_1 = nn.Sequential(
                            ConvBlock(in_channel, out1x1, kernel_size=1)
                        )
        self.branch_2 =  nn.Sequential(
                            ConvBlock(in_channel, re_out3x3, kernel_size=1),
                            ConvBlock(re_out3x3, out3x3, kernel_size=3, padding=1)
                        )
        self.branch_3 =  nn.Sequential(
                            ConvBlock(in_channel, re_out5x5, kernel_size=1, ),
                            ConvBlock(re_out5x5, out5x5, kernel_size=5, padding=2)
                        )
        self.branch_4 = nn.Sequential(
                            nn.MaxPool2d(kernel_size=3, stride=1, padding=1),
                            ConvBlock(in_channel, out_pool, kernel_size=1)
                        )
        self.relu = nn.ReLU()
        
    def forward(self,x):
        
#         b1 = self.branch_1(x)
#         print(b1.shape)
        
#         b2 = self.branch_2(x)
#         print(b2.shape)

#         b3 =  self.branch_3(x)
#         print(b3.shape)

#         b4 =  self.branch_4(x) 
#         print(b4.shape)

        x = torch.cat([self.branch_1(x),
                       self.branch_2(x),
                       self.branch_3(x),
                       self.branch_4(x)
                       ], dim=1)

        return x

        

In [75]:
inception_3a = Inception_Module(192, 64, 96, 128, 16, 32, 32)
# conv1 = ConvBlock(192,64,kernel_size=1)
y = inception_3a(x)
y.shape

torch.Size([2, 256, 28, 28])

In [76]:
64 + 128 + 32 + 32

256

In [110]:
class InceptionV1(nn.Module):
    def __init__(self, in_channels=3, num_classes=1000):
        super(InceptionV1, self).__init__()
        self.conv1 = ConvBlock(in_channels, 64, kernel_size=7, stride=2, padding=3)
        self.maxpool = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)
        self.conv2 = ConvBlock(64, 192, kernel_size=3, stride=1, padding=1)
        self.inception_3a = Inception_Module(192, 64, 96, 128, 16, 32, 32)
        self.inception_3b = Inception_Module(256, 128, 128, 192, 32, 96, 64)
        
        self.inception_4a = Inception_Module(480, 192, 96, 208, 16, 48, 64)
        self.inception_4b = Inception_Module(512, 160, 112, 224, 24, 64, 64)
        self.inception_4c = Inception_Module(512, 128, 128, 256, 24, 64, 64)
        self.inception_4d = Inception_Module(512, 112, 144, 288, 32, 64, 64)
        self.inception_4e = Inception_Module(528, 256, 160, 320, 32, 128, 128)

        self.inception_5a = Inception_Module(832, 256, 160, 320, 32, 128, 128)
        self.inception_5b = Inception_Module(832, 384, 192, 384, 48, 128, 128)

        self.avgpool = nn.AvgPool2d(kernel_size=7, stride=1) #, padding=1)
        
        self.dropout = nn.Dropout(0.4)
        
        self.linear = nn.Linear(1024, num_classes)
        
    def forward(self, x):
        x = self.conv1(x)
        x = self.maxpool(x)
        x = self.conv2(x)
        x = self.maxpool(x)
        
        x = self.inception_3a(x)
        x = self.inception_3b(x)
        
        x = self.maxpool(x)
        
        x = self.inception_4a(x)
        x = self.inception_4b(x)
        x = self.inception_4c(x)
        x = self.inception_4d(x)
        x = self.inception_4e(x)
        
        x = self.maxpool(x)
        
        x = self.inception_5a(x)
        x = self.inception_5b(x)

        x = self.avgpool(x)
        x = self.dropout(x)
        
        x = x.reshape(x.shape[0], -1)
        x = self.linear(x)
        
        return x
        


In [111]:
img_dummy = torch.randn(2, 3, 224, 224)
img_dummy.shape

torch.Size([2, 3, 224, 224])

In [112]:
inception_net = InceptionV1(in_channels=3, num_classes=1000)
y = inception_net(img_dummy)
y.shape

torch.Size([2, 1000])